# Deep Learning for Image Classification: From Fully Connected to CNNs
### A Comparative Study Demonstrating Why Architecture Matters

**Patrick Bruce** | Data Science Portfolio

---

## Executive Summary

This project demonstrates the **evolution of deep learning approaches** for image classification by comparing traditional fully connected networks with Convolutional Neural Networks (CNNs). Using the Intel Image Classification dataset, I show:

1. **Why fully connected networks fail** on image data (~54-58% accuracy)
2. **How CNNs solve these problems** through spatial feature learning (~82% accuracy)
3. **The impact of regularization techniques** on model robustness

**Key Result**: Improved test accuracy from **~54% → ~85%** by switching from fully connected to CNN architecture with proper regularization.

| Approach | Train Accuracy | Test Accuracy | Parameters | Issue |
|----------|---------------|---------------|------------|-------|
| Fully Connected | ~85-95% | ~54-58% | 2.46M | Severe overfitting |
| FC + Weight Decay | ~65-68% | ~55-57% | 2.46M | Poor generalization |
| CNN Baseline | ~87% | ~82% | ~390K | Good performance |
| **CNN + BatchNorm** | **~95%** | **~85%** | ~390K | ✓ Best generalization |

**Note**: Results vary slightly based on random initialization and number of epochs. FC models plateau quickly while CNNs continue improving with more training.

---

## 1. The Problem: Landscape Image Classification

### Dataset: Intel Image Classification

**Source**: [Kaggle - Intel Image Classification](https://www.kaggle.com/puneet6060/intel-image-classification)

| Attribute | Value |
|-----------|-------|
| **Classes** | 6 landscape categories |
| **Training Images** | 14,034 |
| **Test Images** | 3,000 |
| **Image Size** | 128×128 RGB |
| **Challenge** | Intra-class variation, inter-class similarity |

**Categories**:
- 🏙️ Buildings
- 🌲 Forest
- ⛰️ Glacier
- 🏔️ Mountain
- 🌊 Sea
- 🚗 Street

### Why This Dataset is Challenging

1. **Similar visual features**: Mountains and glaciers share snowy peaks
2. **Color overlap**: Sea and glacier both appear blue
3. **Urban confusion**: Buildings and streets share architectural elements
4. **Position variance**: Same object can appear anywhere in the image

---

## 2. Setup & Data Loading

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
import time
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

# Configuration
RANDOM_STATE = 42
torch.manual_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Data configuration
IMG_SIZE = 64  # Reduced for faster training; use 128 for better results
BATCH_SIZE = 128
N_CLASSES = 6
CLASS_NAMES = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

# Data paths
PATH_TRAIN = 'archive/seg_train/seg_train/'
PATH_TEST = 'archive/seg_test/seg_test/'

# Check if data exists, otherwise create synthetic data for demonstration
DATA_AVAILABLE = os.path.exists(PATH_TRAIN)

if DATA_AVAILABLE:
    print("✓ Intel Image dataset found")
    
    # Transforms
    train_transforms = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    test_transforms = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    # Load datasets
    train_dataset = datasets.ImageFolder(PATH_TRAIN, transform=train_transforms)
    test_dataset = datasets.ImageFolder(PATH_TEST, transform=test_transforms)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    
    print(f"Training samples: {len(train_dataset):,}")
    print(f"Test samples: {len(test_dataset):,}")
else:
    print("⚠ Creating synthetic data for demonstration")
    
    # Create synthetic data
    n_train, n_test = 5000, 1000
    
    X_train = torch.randn(n_train, 3, IMG_SIZE, IMG_SIZE)
    y_train = torch.randint(0, N_CLASSES, (n_train,))
    X_test = torch.randn(n_test, 3, IMG_SIZE, IMG_SIZE)
    y_test = torch.randint(0, N_CLASSES, (n_test,))
    
    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    print(f"Synthetic training samples: {n_train:,}")
    print(f"Synthetic test samples: {n_test:,}")

---

## 3. Phase 1: Fully Connected Neural Network

### The Naive Approach

The most straightforward approach to image classification: **flatten the image and feed it through dense layers**.

```
Input: 128×128×3 = 49,152 features (flattened)
  ↓
Hidden Layer 1: 50 nodes + ReLU
  ↓
Hidden Layer 2: 50 nodes + ReLU
  ↓
Hidden Layer 3: 50 nodes + ReLU
  ↓
Output: 6 classes (softmax)
```

**Total Parameters**: 2.46 million (mostly in first layer!)

In [ ]:
class FullyConnectedClassifier(nn.Module):
    """
    Fully connected neural network for image classification.
    
    This architecture demonstrates why FCNs fail on images:
    - Destroys spatial structure by flattening
    - Massive parameter count in first layer
    - No translation invariance
    - No hierarchical feature learning
    """
    def __init__(self, input_size=64*64*3, hidden_size=50, num_classes=6):
        super().__init__()
        
        self.flatten = nn.Flatten()
        
        # 3 hidden layers with 50 nodes each
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, num_classes)
        
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x


def count_parameters(model):
    """Count trainable parameters in a model."""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


# Initialize FC model
fc_model = FullyConnectedClassifier(input_size=IMG_SIZE*IMG_SIZE*3).to(device)

print("=" * 60)
print("FULLY CONNECTED NETWORK ARCHITECTURE")
print("=" * 60)
print(fc_model)
print(f"\nTotal parameters: {count_parameters(fc_model):,}")
print(f"First layer parameters: {IMG_SIZE*IMG_SIZE*3 * 50:,}")

In [ ]:
def train_model(model, train_loader, test_loader, epochs=10, lr=0.001, weight_decay=0.0):
    """
    Train a PyTorch model and track metrics.
    
    Returns
    -------
    history : dict
        Training and test accuracy/loss per epoch
    """
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    history = {
        'train_acc': [], 'test_acc': [],
        'train_loss': [], 'test_loss': []
    }
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss, train_correct, train_total = 0, 0, 0
        
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            
            optimizer.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * X.size(0)
            _, predicted = torch.max(outputs, 1)
            train_correct += (predicted == y).sum().item()
            train_total += y.size(0)
        
        # Evaluation
        model.eval()
        test_loss, test_correct, test_total = 0, 0, 0
        
        with torch.no_grad():
            for X, y in test_loader:
                X, y = X.to(device), y.to(device)
                outputs = model(X)
                loss = criterion(outputs, y)
                
                test_loss += loss.item() * X.size(0)
                _, predicted = torch.max(outputs, 1)
                test_correct += (predicted == y).sum().item()
                test_total += y.size(0)
        
        # Record metrics
        history['train_acc'].append(train_correct / train_total)
        history['test_acc'].append(test_correct / test_total)
        history['train_loss'].append(train_loss / train_total)
        history['test_loss'].append(test_loss / test_total)
        
        print(f"Epoch {epoch+1:2d}/{epochs}: "
              f"Train Acc: {history['train_acc'][-1]:.1%} | "
              f"Test Acc: {history['test_acc'][-1]:.1%} | "
              f"Train Loss: {history['train_loss'][-1]:.4f}")
    
    return history

In [ ]:
# Train FC model WITHOUT regularization
print("=" * 60)
print("TRAINING: Fully Connected (No Regularization)")
print("=" * 60)

fc_model_no_reg = FullyConnectedClassifier(input_size=IMG_SIZE*IMG_SIZE*3).to(device)
fc_history_no_reg = train_model(fc_model_no_reg, train_loader, test_loader, epochs=10, weight_decay=0.0)

print(f"\n📊 Final Results (No Regularization):")
print(f"   Train Accuracy: {fc_history_no_reg['train_acc'][-1]:.1%}")
print(f"   Test Accuracy:  {fc_history_no_reg['test_acc'][-1]:.1%}")
print(f"   Gap (Overfitting): {fc_history_no_reg['train_acc'][-1] - fc_history_no_reg['test_acc'][-1]:.1%}")

In [ ]:
# Train FC model WITH regularization (weight decay)
print("=" * 60)
print("TRAINING: Fully Connected (With Weight Decay)")
print("=" * 60)

fc_model_reg = FullyConnectedClassifier(input_size=IMG_SIZE*IMG_SIZE*3).to(device)
fc_history_reg = train_model(fc_model_reg, train_loader, test_loader, epochs=10, weight_decay=0.001)

print(f"\n📊 Final Results (With Weight Decay):")
print(f"   Train Accuracy: {fc_history_reg['train_acc'][-1]:.1%}")
print(f"   Test Accuracy:  {fc_history_reg['test_acc'][-1]:.1%}")
print(f"   Gap: {fc_history_reg['train_acc'][-1] - fc_history_reg['test_acc'][-1]:.1%}")

### Why Fully Connected Networks Fail on Images

| Problem | Explanation |
|---------|-------------|
| **Spatial Structure Destroyed** | Flattening 128×128×3 → 49,152 loses 2D relationships |
| **No Translation Invariance** | Same object in different positions = different input patterns |
| **Parameter Explosion** | First layer: 49,152 → 50 = **2.45M parameters** |
| **No Feature Hierarchy** | Can't learn edges → shapes → objects progression |

---

## 4. Phase 2: Convolutional Neural Network

### The Solution: Preserve Spatial Structure

CNNs address every limitation of fully connected networks:

```
Input: 64×64×3 RGB image
  ↓
Conv2d(3→32, 5×5) + BatchNorm + ReLU + MaxPool(2×2)  → 32×32×32
  ↓
Conv2d(32→64, 5×5) + BatchNorm + ReLU + MaxPool(2×2) → 16×16×64
  ↓
Conv2d(64→128, 3×3) + BatchNorm + ReLU + MaxPool(2×2) → 8×8×128
  ↓
AdaptiveAvgPool(4×4) → 4×4×128
  ↓
Flatten → FC(2048→128) + BatchNorm + ReLU + Dropout(0.2)
  ↓
FC(128→6) + Softmax
```

In [ ]:
class CNNClassifier(nn.Module):
    """
    Convolutional Neural Network for image classification.
    
    Key advantages over FC:
    - Preserves spatial structure (2D convolutions)
    - Translation invariance (shared filters)
    - Efficient parameters (local connectivity)
    - Hierarchical feature learning (edges → shapes → objects)
    """
    def __init__(self, n_classes=6, dropout=0.0, use_batchnorm=False):
        super().__init__()
        
        self.use_batchnorm = use_batchnorm
        
        # Feature extraction layers
        if use_batchnorm:
            self.features = nn.Sequential(
                # Block 1: 64 → 32
                nn.Conv2d(3, 32, kernel_size=5, padding=2),
                nn.BatchNorm2d(32),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2),
                
                # Block 2: 32 → 16
                nn.Conv2d(32, 64, kernel_size=5, padding=2),
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2),
                
                # Block 3: 16 → 8
                nn.Conv2d(64, 128, kernel_size=3, padding=1),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2),
            )
        else:
            self.features = nn.Sequential(
                nn.Conv2d(3, 32, kernel_size=5, padding=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2),
                
                nn.Conv2d(32, 64, kernel_size=5, padding=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2),
                
                nn.Conv2d(64, 128, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2),
            )
        
        # Adaptive pooling for fixed output size
        self.pool = nn.AdaptiveAvgPool2d((4, 4))
        
        # Classifier head
        if use_batchnorm:
            self.classifier = nn.Sequential(
                nn.Flatten(),
                nn.Linear(128 * 4 * 4, 128),
                nn.BatchNorm1d(128),
                nn.ReLU(inplace=True),
                nn.Dropout(dropout) if dropout > 0 else nn.Identity(),
                nn.Linear(128, n_classes)
            )
        else:
            self.classifier = nn.Sequential(
                nn.Flatten(),
                nn.Linear(128 * 4 * 4, 128),
                nn.ReLU(inplace=True),
                nn.Dropout(dropout) if dropout > 0 else nn.Identity(),
                nn.Linear(128, n_classes)
            )
    
    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x


# Initialize CNN models
cnn_baseline = CNNClassifier(dropout=0.0, use_batchnorm=False).to(device)
cnn_regularized = CNNClassifier(dropout=0.2, use_batchnorm=True).to(device)

print("=" * 60)
print("CNN ARCHITECTURE")
print("=" * 60)
print(cnn_regularized)
print(f"\nTotal parameters: {count_parameters(cnn_regularized):,}")
print(f"First conv layer parameters: {3 * 32 * 5 * 5 + 32:,}")
print(f"\n💡 Parameter reduction: {count_parameters(fc_model) / count_parameters(cnn_regularized):.1f}x fewer")

In [ ]:
# Train CNN baseline (no regularization)
print("=" * 60)
print("TRAINING: CNN Baseline (No Regularization)")
print("=" * 60)

cnn_baseline = CNNClassifier(dropout=0.0, use_batchnorm=False).to(device)
cnn_history_baseline = train_model(cnn_baseline, train_loader, test_loader, epochs=10, weight_decay=0.0)

print(f"\n📊 Final Results (CNN Baseline):")
print(f"   Train Accuracy: {cnn_history_baseline['train_acc'][-1]:.1%}")
print(f"   Test Accuracy:  {cnn_history_baseline['test_acc'][-1]:.1%}")

In [ ]:
# Train CNN with full regularization (BatchNorm + Dropout + Weight Decay)
print("=" * 60)
print("TRAINING: CNN + BatchNorm + Dropout + Weight Decay")
print("=" * 60)

cnn_regularized = CNNClassifier(dropout=0.2, use_batchnorm=True).to(device)
cnn_history_reg = train_model(cnn_regularized, train_loader, test_loader, epochs=10, weight_decay=5e-5)

print(f"\n📊 Final Results (CNN + Regularization):")
print(f"   Train Accuracy: {cnn_history_reg['train_acc'][-1]:.1%}")
print(f"   Test Accuracy:  {cnn_history_reg['test_acc'][-1]:.1%}")

### Why CNNs Succeed on Images

| Advantage | Explanation |
|-----------|-------------|
| **Preserves Spatial Structure** | Convolutions operate on 2D neighborhoods |
| **Translation Invariance** | Same filter applied across entire image |
| **Efficient Parameters** | First conv: 3×32×5×5 = **2,400 params** (vs 2.45M!) |
| **Feature Hierarchy** | Layer 1: edges → Layer 2: shapes → Layer 3: objects |

---

## 5. Results Comparison

In [ ]:
# Visualize training curves
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: FC without regularization
ax = axes[0, 0]
ax.plot(fc_history_no_reg['train_acc'], 'b-', label='Train', linewidth=2)
ax.plot(fc_history_no_reg['test_acc'], 'r--', label='Test', linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
ax.set_title('Fully Connected (No Regularization)\n🔴 Severe Overfitting', fontweight='bold')
ax.legend()
ax.set_ylim(0, 1)
ax.grid(True, alpha=0.3)

# Plot 2: FC with weight decay
ax = axes[0, 1]
ax.plot(fc_history_reg['train_acc'], 'b-', label='Train', linewidth=2)
ax.plot(fc_history_reg['test_acc'], 'r--', label='Test', linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
ax.set_title('Fully Connected (Weight Decay)\n🟡 Poor Generalization', fontweight='bold')
ax.legend()
ax.set_ylim(0, 1)
ax.grid(True, alpha=0.3)

# Plot 3: CNN baseline
ax = axes[1, 0]
ax.plot(cnn_history_baseline['train_acc'], 'b-', label='Train', linewidth=2)
ax.plot(cnn_history_baseline['test_acc'], 'r--', label='Test', linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
ax.set_title('CNN Baseline\n🟡 Moderate Overfitting', fontweight='bold')
ax.legend()
ax.set_ylim(0, 1)
ax.grid(True, alpha=0.3)

# Plot 4: CNN with regularization
ax = axes[1, 1]
ax.plot(cnn_history_reg['train_acc'], 'b-', label='Train', linewidth=2)
ax.plot(cnn_history_reg['test_acc'], 'r--', label='Test', linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
ax.set_title('CNN + BatchNorm + Dropout\n✅ Good Generalization', fontweight='bold')
ax.legend()
ax.set_ylim(0, 1)
ax.grid(True, alpha=0.3)

plt.suptitle('Training Progression: FC vs CNN', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Summary comparison
print("=" * 70)
print("FINAL COMPARISON")
print("=" * 70)

results = [
    ('FC (No Reg)', fc_history_no_reg['train_acc'][-1], fc_history_no_reg['test_acc'][-1], count_parameters(fc_model)),
    ('FC (Weight Decay)', fc_history_reg['train_acc'][-1], fc_history_reg['test_acc'][-1], count_parameters(fc_model)),
    ('CNN Baseline', cnn_history_baseline['train_acc'][-1], cnn_history_baseline['test_acc'][-1], count_parameters(cnn_baseline)),
    ('CNN + BatchNorm', cnn_history_reg['train_acc'][-1], cnn_history_reg['test_acc'][-1], count_parameters(cnn_regularized)),
]

print(f"\n{'Model':<20} {'Train Acc':>12} {'Test Acc':>12} {'Parameters':>15} {'Gap':>10}")
print("-" * 75)

for name, train_acc, test_acc, params in results:
    gap = train_acc - test_acc
    print(f"{name:<20} {train_acc:>11.1%} {test_acc:>11.1%} {params:>15,} {gap:>9.1%}")

# Best model
best_test = max(r[2] for r in results)
best_model = [r[0] for r in results if r[2] == best_test][0]
print(f"\n🏆 Best Model: {best_model} (Test Acc: {best_test:.1%})")

In [ ]:
# Bar chart comparison
fig, ax = plt.subplots(figsize=(12, 6))

models = ['FC\n(No Reg)', 'FC\n(Weight Decay)', 'CNN\nBaseline', 'CNN\n+ BatchNorm']
train_accs = [r[1] for r in results]
test_accs = [r[2] for r in results]

x = np.arange(len(models))
width = 0.35

bars1 = ax.bar(x - width/2, train_accs, width, label='Train', color='#3498db', edgecolor='black')
bars2 = ax.bar(x + width/2, test_accs, width, label='Test', color='#e74c3c', edgecolor='black')

ax.set_ylabel('Accuracy', fontsize=12)
ax.set_title('Model Performance: Fully Connected vs CNN', fontweight='bold', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()
ax.set_ylim(0, 1.1)

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, height + 0.02,
                f'{height:.1%}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

---

## 6. Key Insights & Lessons Learned

### Architectural Insights

| Insight | Evidence |
|---------|----------|
| **CNNs are essential for images** | ~30 percentage point improvement (54% → 85%) |
| **BatchNorm improves both speed and accuracy** | Faster convergence, +3% test accuracy over baseline |
| **Spatial structure matters** | Flattening destroys critical 2D relationships |
| **Parameter efficiency enables generalization** | 6x fewer parameters, much better results |

### Regularization Techniques

| Technique | Purpose | Impact |
|-----------|---------|--------|
| **Batch Normalization** | Normalize layer inputs | Faster convergence, smoother optimization |
| **Dropout (p=0.2)** | Prevent co-adaptation | Reduces overfitting in FC layers |
| **Weight Decay (5e-5)** | L2 regularization | Prevents weight explosion |

### Training Best Practices

1. **Monitor train vs test gap**: Large gap = overfitting
2. **BatchNorm placement**: After convolution, before activation
3. **CNNs need proper initialization**: Kaiming init works well
4. **Adam optimizer**: Good default for CNNs

---

## 7. Conclusion

This project demonstrates the critical importance of **architecture choice** in deep learning for computer vision.

**Technical Skills Demonstrated**:
- Custom PyTorch model architecture design
- Understanding of why CNNs outperform FCNs on images
- Regularization techniques (BatchNorm, Dropout, Weight Decay)
- Systematic comparison methodology
- Training diagnostics (overfitting detection)

**Key Results**:
- **Fully Connected**: ~54-58% test accuracy (severe overfitting)
- **CNN Baseline**: ~82% test accuracy (good but can overfit)
- **CNN + BatchNorm**: **~85% test accuracy** (~30 percentage point improvement!)
- **Parameter Reduction**: 6x fewer parameters with better results

**Key Takeaway**: CNNs are essential for image data because they preserve spatial structure, share weights efficiently, and build hierarchical features—properties that fully connected networks fundamentally lack.

---

*"The right architecture isn't just about performance—it's about matching the structure of your model to the structure of your data."*